# practice unsupervised learning

##### in this notebook we will practice and explore unsupervised learning techniques
##### in each section we will explore a different technique, and have a practical code example
##### the steps we will follow are:
##### 1. load the data
##### 2. explore and visualized the data
##### 3. preprocess the data
##### 4. evaluate the performance of some clustering algorithms
##### 5. visualize the clusters
##### 6. evaluate the performance of some dimensionality reduction algorithms
##### 7. visualize the data in 2D
##### 8. visualize the data in 3D


In [3]:
# step 1. import all the required libraries and load the data